In [10]:
import numpy as np
from qiskit import *
from qiskit_ionq_provider import IonQProvider 
from qiskit.providers.jobstatus import JobStatus
#Call provider and set token value
provider = IonQProvider(token='BFmvdArkiCbsS12r4LZf5VgYDo4HngsS')
from qiskit_textbook.tools import array_to_latex

from signature import *
from hash import crypto_hash
from swap_test import swap_test

In [11]:
state = crypto_hash('1001', ['1011', '1101', '1001', '0010'], verbose=True)

         ┌───┐                                                            »
q4481_0: ┤ H ├──────o────────────■───────────o───────────■──────────o─────»
         ├───┤      │            │           │           │          │     »
q4481_1: ┤ H ├──────o────────────o───────────■───────────■──────────o─────»
         └───┘┌─────┴─────┐┌─────┴─────┐┌────┴─────┐┌────┴────┐┌────┴────┐»
q4481_2: ─────┤ RY(11π/4) ├┤ RY(13π/4) ├┤ RY(9π/4) ├┤ RY(π/2) ├┤ RY(22π) ├»
              └───────────┘└───────────┘└──────────┘└─────────┘└─────────┘»
«                                         
«q4481_0: ─────■──────────o─────────■─────
«              │          │         │     
«q4481_1: ─────o──────────■─────────■─────
«         ┌────┴────┐┌────┴────┐┌───┴────┐
«q4481_2: ┤ RY(26π) ├┤ RY(18π) ├┤ RY(4π) ├
«         └─────────┘└─────────┘└────────┘


In [12]:
array_to_latex(state, pretext="\\text{Hash('1001')} = ")

<IPython.core.display.Math object>

In [13]:
signed_message = Signed_Transaction("1234556")
forged_message = Forgery("1234556")

In [14]:
good_validation = Validation(signed_message)
print(good_validation.validate())

bad_validation = Validation(forged_message)
print(bad_validation.validate())

{'pass': True, 'bits_failure': '0.0%'}
{'pass': False, 'bits_failure': '71.4286%'}


In [5]:
# passes = 0
# fails = 0

# for i in range(5):
#     # Message bit
#     b = 0

#     sig = Signature(b)
#     sig.get_signature()['pub_keys']['f_k1']
#     val = Validation(sig)
    
    
#     result = val.result()

#     if result['passed']:
#         passes += 1
#     else:
#         fails += 1

# print(passes, fails)

In [28]:
pub_0 = signed_message.get_signature()[0]['pub_keys']['f_k0'][0]
pub_1 = signed_message.get_signature()[0]['pub_keys']['f_k0'][0]

swap_test(pub_0, pub_1, verbose=True)

                               ┌───┐                                          »
q4530_0: ──────────────────────┤ H ├──────────────────────────────────■───────»
         ┌─────────────────────┴───┴─────────────────────┐┌───┐       │  ┌───┐»
q4530_1: ┤0                                              ├┤ X ├───────■──┤ X ├»
         │  initialize(0.65328,-0.65328,-0.2706,-0.2706) │└─┬─┘┌───┐  │  └─┬─┘»
q4530_2: ┤1                                              ├──┼──┤ X ├──┼────┼──»
         ├───────────────────────────────────────────────┤  │  └─┬─┘┌─┴─┐  │  »
q4530_3: ┤0                                              ├──■────┼──┤ X ├──■──»
         │  initialize(0.65328,-0.65328,-0.2706,-0.2706) │       │  └───┘     »
q4530_4: ┤1                                              ├───────■────────────»
         └───────────────────────────────────────────────┘                    »
 c269: 1/═════════════════════════════════════════════════════════════════════»
                                        

0

<IPython.core.display.Math object>